In [1]:
import numpy as np

In [2]:
def process_text(filename):
    with open(filename, 'r') as file:
        data = file.read()
    chars = list(set(data))
    char_to_ix = {ch: i for i, ch in enumerate(chars)}
    ix_to_char = {i: ch for i, ch in enumerate(chars)}
    return data, char_to_ix, ix_to_char

In [3]:
def initialize_parameters(hidden_size, vocab_size):
    Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
    Whh = np.random.randn(hidden_size, hidden_size) * 0.01
    Why = np.random.randn(vocab_size, hidden_size) * 0.01
    bh = np.zeros((hidden_size, 1))
    by = np.zeros((vocab_size, 1))
    return Wxh, Whh, Why, bh, by

In [4]:
def rnn_forward(inputs, targets, hprev, Wxh, Whh, Why, bh, by):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0

    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0])

    return loss, xs, hs, ys, ps

In [5]:
def rnn_backward(inputs, targets, xs, hs, ys, ps, Wxh, Whh, Why, bh, by):
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)

    return dWxh, dWhh, dWhy, dbh, dby

In [6]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []

    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)

    return ixes

In [7]:
data, char_to_ix, ix_to_char = process_text('alice_in_wonderland.txt')

hidden_size = 100
vocab_size = len(char_to_ix)
seq_length = 25
learning_rate = 1e-1

Wxh, Whh, Why, bh, by = initialize_parameters(hidden_size, vocab_size)

In [8]:
def lossFun(inputs, targets, hprev):
    loss, xs, hs, ys, ps = rnn_forward(inputs, targets, hprev, Wxh, Whh, Why, bh, by)
    gradients = rnn_backward(inputs, targets, xs, hs, ys, ps, Wxh, Whh, Why, bh, by)
    return loss, *gradients, hs[len(inputs) - 1]

In [9]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0 / vocab_size) * seq_length

In [10]:
for _ in range(500):
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print(f'\n\n\n\n {txt}')

    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print(f'iter #{n}, loss: {smooth_loss}')

    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

    p += seq_length
    n += 1





 2O)4e@jUyK*aHCdfWL 2DxQZ[FNC6FbKbhyb=NXGR15_96BSHMwWj
2!ipXAv@Tra-`CAS6@P!5COL!6vYKH
MHi"1bL)nIB?H6Vd3Mk"='Ut4t931?]mF3;PW;6,eT!'NbU_VMd68 )!U?
"cwk9oj"l9FX[4d5D;]8](1Z?,u=tRIc1THeLT0qW)YO12aV3m5tG*
I
iter #0, loss: 109.86122216810888




  `l l ellakdellA srt  l.eeo 
p `ae retae lelkolmilhesait ila lpelsr ee uetyep stlec esdotlnl  sWe vesg  h ` is edOessend s  t e
a e etweloulheame
oe  Tmt l en, lwe
aer tpaethelW narnaC`  ll tlelsellit
iter #100, loss: 109.4489201756467




 o `a  danaaPu  sh
,irtRatWrhy  gidw ug t saltnoa hndOweo ra
sy
tweaafe `taiIr ,einaahaudh wyktAt  i e
arinVfrwuhvwd a,a, otho  dirt hnstasnnAna nivtq`t spmeatita , uh' acdrt
-ewhcstor y
tg-wyih f  eni
iter #200, loss: 107.10096071122908




 t ateir filo:sse )p  ln dt v s on weo keme:o rhj d tu ilte
nnc rrlh kg`gele segueaoitowehherin
we
on  tloy adoflr dmh,thhn di ole !e Hn telkee'nt fgnft
o rwwognrynnkor?r a iatoS tblent awsels go e
ogi
iter #300, loss: 104.35490316677425




 ikeseyciw   wglrt nhore s  h
i